# COVID-19 Анализ: Глобальные тренды

В этом ноутбуке анализируем как развивается COVID в разных странах

Данные: https://www.kaggle.com/datasets/imdevskp/corona-virus-report

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# устанавливаем стили для графиков
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print('Все библиотеки загружены!')

## Загружаем данные

In [ ]:
# скачиваем датасет с Kaggle
# можно использовать wget или просто скачать вручную

# я буду создавать синтетические данные для примера
# так как не всегда есть доступ к интернету

from datetime import timedelta

# создаем временной диапазон
dates = pd.date_range(start='2020-03-01', end='2023-12-31', freq='D')

# список стран (выбрал интересные для анализа)
countries = ['Russia', 'USA', 'Germany', 'Italy', 'Brazil', 'India', 'China', 'UK', 'France', 'Spain']

# создаем данные
data_list = []

for country in countries:
    for date in dates:
        # каждая страна имеет свой паттерн
        days_since_start = (date - pd.Timestamp('2020-03-01')).days
        
        # волны заболеваемости (примерно)
        if country == 'Russia':
            cases = int(100 * np.exp(0.03 * days_since_start) * (1 + 0.3 * np.sin(days_since_start/100)))
            deaths = int(cases * 0.02)
        elif country == 'USA':
            cases = int(150 * np.exp(0.032 * days_since_start) * (1 + 0.4 * np.sin(days_since_start/80)))
            deaths = int(cases * 0.015)
        elif country == 'India':
            cases = int(200 * np.exp(0.035 * days_since_start) * (1 + 0.5 * np.sin(days_since_start/120)))
            deaths = int(cases * 0.01)
        else:
            # для остальных стран просто случайные значения с трендом
            cases = int(50 * np.exp(0.025 * days_since_start) * np.random.uniform(0.8, 1.2))
            deaths = int(cases * np.random.uniform(0.01, 0.025))
        
        recovered = int(cases * 0.95)
        
        data_list.append({
            'Date': date,
            'Country': country,
            'Confirmed': max(0, cases),
            'Deaths': max(0, deaths),
            'Recovered': max(0, recovered)
        })

df = pd.DataFrame(data_list)
print(f'Данные загружены! Форма: {df.shape}')
print(df.head(10))

## Исследовательский анализ (EDA)

In [ ]:
# смотрим основную инфу о данных
print('Информация о датасете:')
print(df.info())
print('\nОсновные статистики:')
print(df.describe())
print('\nЕсть ли пропуски?')
print(df.isnull().sum())

In [ ]:
# считаем общую статистику по странам
country_stats = df.groupby('Country')[['Confirmed', 'Deaths', 'Recovered']].sum().sort_values('Confirmed', ascending=False)
print('Всего случаев по странам:')
print(country_stats)

# смертность по странам
country_stats['Mortality_Rate'] = (country_stats['Deaths'] / country_stats['Confirmed'] * 100).round(2)
print('\nСмертность %:')
print(country_stats[['Mortality_Rate']].sort_values('Mortality_Rate', ascending=False))

## Графики

In [ ]:
# График: Всего случаев по странам (за весь период)
fig, ax = plt.subplots(figsize=(12, 6))

country_totals = df.groupby('Country')['Confirmed'].sum().sort_values(ascending=True)
country_totals.plot(kind='barh', ax=ax, color='steelblue')

ax.set_xlabel('Всего случаев')
ax.set_ylabel('Страна')
ax.set_title('Всего подтверждённых случаев COVID-19 по странам (март 2020 - декабрь 2023)')

plt.tight_layout()
plt.savefig('plots/01_cases_by_country.png', dpi=300, bbox_inches='tight')
plt.show()

print('График сохранён!')

In [ ]:
# Временный ряд: как менялись случаи во времени
fig, ax = plt.subplots(figsize=(14, 7))

# берем топ 5 стран по количеству случаев
top_countries = df.groupby('Country')['Confirmed'].sum().nlargest(5).index

for country in top_countries:
    country_data = df[df['Country'] == country].sort_values('Date')
    ax.plot(country_data['Date'], country_data['Confirmed'], label=country, linewidth=2)

ax.set_xlabel('Дата')
ax.set_ylabel('Количество случаев')
ax.set_title('COVID-19: Волны заболеваемости в разных странах')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plots/02_timeline_cases.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# График смертности
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# абсолютные числа
deaths_by_country = df.groupby('Country')['Deaths'].sum().sort_values(ascending=False)
deaths_by_country.plot(kind='bar', ax=ax1, color='crimson')
ax1.set_title('Всего смертей по странам')
ax1.set_ylabel('Количество смертей')
ax1.tick_params(axis='x', rotation=45)

# процент смертности
mortality = (df.groupby('Country')['Deaths'].sum() / df.groupby('Country')['Confirmed'].sum() * 100).sort_values(ascending=False)
mortality.plot(kind='bar', ax=ax2, color='orange')
ax2.set_title('Процент смертности')
ax2.set_ylabel('% смертности')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('plots/03_mortality_stats.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Распределение случаев (гистограмма)
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].hist(df['Confirmed'], bins=50, color='steelblue', edgecolor='black')
axes[0, 0].set_title('Распределение случаев')
axes[0, 0].set_xlabel('Количество случаев')

axes[0, 1].hist(df['Deaths'], bins=50, color='crimson', edgecolor='black')
axes[0, 1].set_title('Распределение смертей')
axes[0, 1].set_xlabel('Количество смертей')

axes[1, 0].hist(df['Recovered'], bins=50, color='green', edgecolor='black')
axes[1, 0].set_title('Распределение выздоровевших')
axes[1, 0].set_xlabel('Количество выздоровевших')

# корреляция
corr_matrix = df[['Confirmed', 'Deaths', 'Recovered']].corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', ax=axes[1, 1])
axes[1, 1].set_title('Корреляция между показателями')

plt.tight_layout()
plt.savefig('plots/04_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# сохраняем обработанные данные для следующего ноутбука
df.to_csv('data/covid_clean.csv', index=False)
print('Данные сохранены в data/covid_clean.csv')
print(f'\nВсего строк: {len(df)}')
print(f'Дата начала: {df["Date"].min()}')
print(f'Дата конца: {df["Date"].max()}')
print(f'Страны: {df["Country"].nunique()}')